In [1]:
import numpy as np
from pathlib import Path

combined_data_filenames = {'Base':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\Non_capped\trb_paper_base_combined.stats.npz'),
                          'No_shift':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\Non_capped\trb_paper_no_shifts_combined.stats.npz'),
                          'Longer':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\Non_capped\trb_paper_longer_visit_combined.stats.npz'),
                          'No_neighbourhood':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\Non_capped\trb_paper_no_neighbourhoods_combined.stats.npz')}

combined_data = {label:np.load(combined_data_filenames[label]) for label, filename in combined_data_filenames.items()} 

                 
                 

In [2]:
data_names_small = (
    '1.0_staff_customer_weight_over_contacts',
    '1.0_staff_customer_weight_over_agents',
    '1.0_staff_customer_contacts_over_agents',
    None,
    '1.0_customer_staff_weight_over_agents',
    '1.0_customer_staff_contacts_over_agents',
    '1.0_customer_customer_weight_over_contacts',
    '1.0_customer_customer_weight_over_agents',
    '1.0_customer_customer_contacts_over_agents',
)

data_names_middle = (
    '1.5_staff_customer_weight_over_contacts',
    '1.5_staff_customer_weight_over_agents',
    '1.5_staff_customer_contacts_over_agents',
    None,
    '1.5_customer_staff_weight_over_agents',
    '1.5_customer_staff_contacts_over_agents',
    '1.5_customer_customer_weight_over_contacts',
    '1.5_customer_customer_weight_over_agents',
    '1.5_customer_customer_contacts_over_agents'
)

data_names_large = (
    '2.0_staff_customer_weight_over_contacts',
    '2.0_staff_customer_weight_over_agents',
    '2.0_staff_customer_contacts_over_agents',
    None,
    '2.0_customer_staff_weight_over_agents',
    '2.0_customer_staff_contacts_over_agents',
    '2.0_customer_customer_weight_over_contacts',    
    '2.0_customer_customer_weight_over_agents',
    '2.0_customer_customer_contacts_over_agents'
)

data_name_groups = {data_names_small:'1.0', 
                    data_names_middle:'1.5', 
                    data_names_large:'2.0'}



In [3]:
from contact_plots import get_cdf_of_data

%matplotlib widget
import matplotlib.pyplot as plt

row_count = 3
col_count = 3

fig, axes = plt.subplots(row_count, col_count)
axes = axes.flatten()

colors = ('tab:red', 'tab:blue', 'tab:green', 'tab:orange')
line_styles = ('-', '--', ':')


line_style_ind = 0

max_x = {0:0,1:0,2:0}

legend_ax = None
legend_content = {}
label_2_color_ind = {}

for data_names, dist_label in data_name_groups.items():
    ax_ind = 0
    line_style = line_styles[line_style_ind]
    for data_name in data_names:
        if data_name is None:
            legend_ax = axes[ax_ind]
            ax_ind += 1
            continue
        color_ind = 0
        for label, data in combined_data.items():
            if label not in label_2_color_ind:
                label_2_color_ind[label] = color_ind
                color_ind += 1                
            color = colors[label_2_color_ind[label]]

            if len(data[data_name]) == 0:
                continue
            x, y = get_cdf_of_data(data[data_name])
            col_ind = ax_ind % col_count
            max_x[col_ind] = max(max_x[col_ind], max(x))
            line_handle = axes[ax_ind].plot(x, y, color=color, linestyle=line_style, label=f'{dist_label} - {label}')
            print(f'{dist_label} - {label} - {data_name} - {len(y)}')
            if (color, line_style) not in legend_content:
                legend_content[(color, line_style)] = (line_handle[0], f'{dist_label} - {label}')
            
            
        axes[ax_ind].set_title(data_name[4:])
        ax_ind += 1
    line_style_ind += 1

for ax_ind in range(row_count*col_count):
    col_ind = ax_ind % col_count
    axes[ax_ind].set_xlim(-max_x[col_ind]*0.05, max_x[col_ind])
    
    
legend_handles = [value[0] for value in legend_content.values()]
legend_labels = [value[1] for value in legend_content.values()]

if legend_ax is not None:
    leg = legend_ax.legend(legend_handles, legend_labels)
else:
    leg = plt.legend(legend_handles, legend_labels)
leg.set_draggable(True)
    
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0 - Base - 1.0_staff_customer_weight_over_contacts - 25588
1.0 - No_shift - 1.0_staff_customer_weight_over_contacts - 26167
1.0 - Longer - 1.0_staff_customer_weight_over_contacts - 19989
1.0 - No_neighbourhood - 1.0_staff_customer_weight_over_contacts - 37564
1.0 - Base - 1.0_staff_customer_weight_over_agents - 360
1.0 - No_shift - 1.0_staff_customer_weight_over_agents - 360
1.0 - Longer - 1.0_staff_customer_weight_over_agents - 375
1.0 - No_neighbourhood - 1.0_staff_customer_weight_over_agents - 345
1.0 - Base - 1.0_staff_customer_contacts_over_agents - 360
1.0 - No_shift - 1.0_staff_customer_contacts_over_agents - 360
1.0 - Longer - 1.0_staff_customer_contacts_over_agents - 375
1.0 - No_neighbourhood - 1.0_staff_customer_contacts_over_agents - 345
1.0 - Base - 1.0_customer_staff_weight_over_agents - 15562
1.0 - No_shift - 1.0_customer_staff_weight_over_agents - 15203
1.0 - Longer - 1.0_customer_staff_weight_over_agents - 11232
1.0 - No_neighbourhood - 1.0_customer_staff_weight_over

In [4]:
print(0 % 3)
print(1 % 3)
print(2 % 3)
print(3 % 3)
print(4 % 3)


0
1
2
0
1


In [18]:
%matplotlib widget
import matplotlib.pyplot as plt

from _collections import defaultdict
from math import ceil

import numpy as np

from math import inf

STAFF_CUSTOMER = 'staff_customer'
CUSTOMER_CUSTOMER = 'customer_customer'

interval_start = 0
interval_step = 1 #s   
max_interval = interval_step*60 #s
bins = np.arange(interval_start,max_interval, interval_step).tolist()
bins.append(max_interval + interval_step)
hist_data = defaultdict(list)       
    
labels = defaultdict(list)
    
for data_names, dist_label in data_name_groups.items():
    for data_name in data_names:
        if data_name is None or not 'weight_over_contacts' in data_name:
             continue
        if STAFF_CUSTOMER in data_name:              
            interaction_type = STAFF_CUSTOMER
        else:
            interaction_type = CUSTOMER_CUSTOMER                       
        for label, data in combined_data.items():
            contact_data = data[data_name]
            contact_data[contact_data>bins[-1]] = bins[-1]
            
            hist_data[(interaction_type, dist_label)].append(contact_data)
            labels[(interaction_type, dist_label)].append(label)
                    
fig, axes = plt.subplots(len(hist_data), 1)
axes = axes.flatten()

ax_ind = 0
for key, hist_data_label in hist_data.items():
    axes[ax_ind].hist(hist_data_label, bins, label=labels[key])
    axes[ax_ind].set_title(f'{key[0]} - {key[1]}')
    ax_ind += 1
    
leg = plt.legend()
leg.set_draggable(True)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
from contact_plots import get_cdf_of_data

%matplotlib widget
import matplotlib.pyplot as plt

row_count = 3
col_count = 1

fig, axes = plt.subplots(row_count, col_count)
axes = axes.flatten()

colors = ('tab:red', 'tab:blue', 'tab:green', 'tab:orange')
line_styles = ('-', '--', ':')


line_style_ind = 0

max_x = {0:0,1:0,2:0}

legend_content = {}
label_2_color_ind = {}

for data_names, dist_label in data_name_groups.items():
    ax_ind = 0
    line_style = line_styles[line_style_ind]
    for data_name in data_names:
        if data_name is None or 'weight_over_contacts' not in data_name:
            continue
        color_ind = 0
        for label, data in combined_data.items():
            if label not in label_2_color_ind:
                label_2_color_ind[label] = color_ind
                color_ind += 1                
            color = colors[label_2_color_ind[label]]
           
            if len(data[data_name]) == 0:
                continue
            x, y = get_cdf_of_data(data[data_name])
            col_ind = ax_ind % col_count
            max_x[col_ind] = max(max_x[col_ind], max(x))
            line_handle = axes[ax_ind].plot(x, y, color=color, linestyle=line_style, label=f'{dist_label} - {label}')
            #print(line_handle)
            if (color, line_style) not in legend_content:
                legend_content[(color, line_style)] = (line_handle[0], f'{dist_label} - {label}')

        axes[ax_ind].set_title(data_name[4:])
        ax_ind += 1
    line_style_ind += 1

legend_ax = axes[-1]    
    
for ax_ind in range(row_count*col_count):
    col_ind = ax_ind % col_count
    axes[ax_ind].set_xlim(-max_x[col_ind]*0.05, max_x[col_ind])
    
legend_handles = [value[0] for value in legend_content.values()]
legend_labels = [value[1] for value in legend_content.values()]

if legend_ax is not None:
    leg = legend_ax.legend(legend_handles, legend_labels)
else:
    leg = plt.legend(legend_handles, legend_labels)
leg.set_draggable(True)
    
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
from contact_plots import get_cdf_of_data

%matplotlib widget
import matplotlib.pyplot as plt

row_count = 3
col_count = 1

fig, axes = plt.subplots(row_count, col_count)
axes = axes.flatten()

colors = ('tab:red', 'tab:blue', 'tab:green', 'tab:orange')
line_styles = ('-', '--', ':')


line_style_ind = 0

max_x = {0:0,1:0,2:0}

legend_content = {}
label_2_color_ind = {}

for data_names, dist_label in data_name_groups.items():
    ax_ind = 0
    line_style = line_styles[line_style_ind]
    for data_name in data_names:
        if data_name is None or 'customer_customer' not in data_name:
            continue
        color_ind = 0
        for label, data in combined_data.items():
            if label not in label_2_color_ind:
                label_2_color_ind[label] = color_ind
                color_ind += 1                
            color = colors[label_2_color_ind[label]]
           
            if len(data[data_name]) == 0:
                continue
            x, y = get_cdf_of_data(data[data_name])
            col_ind = ax_ind % col_count
            max_x[col_ind] = max(max_x[col_ind], max(x))
            line_handle = axes[ax_ind].plot(x, y, color=color, linestyle=line_style, label=f'{dist_label} - {label}')
            print(f'{dist_label} - {label} - {data_name} - {len(y)}')
            if (color, line_style) not in legend_content:
                legend_content[(color, line_style)] = (line_handle[0], f'{dist_label} - {label}')

        axes[ax_ind].set_title(data_name[4:])
        ax_ind += 1
    line_style_ind += 1

legend_ax = axes[-1]    
    
   
legend_handles = [value[0] for value in legend_content.values()]
legend_labels = [value[1] for value in legend_content.values()]

if legend_ax is not None:
    leg = legend_ax.legend(legend_handles, legend_labels)
else:
    leg = plt.legend(legend_handles, legend_labels)
leg.set_draggable(True)
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0 - Base - 1.0_customer_customer_weight_over_contacts - 15
1.0 - Longer - 1.0_customer_customer_weight_over_contacts - 13
1.0 - No_neighbourhood - 1.0_customer_customer_weight_over_contacts - 29
1.0 - Base - 1.0_customer_customer_weight_over_agents - 28
1.0 - Longer - 1.0_customer_customer_weight_over_agents - 25
1.0 - No_neighbourhood - 1.0_customer_customer_weight_over_agents - 53
1.0 - Base - 1.0_customer_customer_contacts_over_agents - 28
1.0 - Longer - 1.0_customer_customer_contacts_over_agents - 25
1.0 - No_neighbourhood - 1.0_customer_customer_contacts_over_agents - 53
1.5 - Base - 1.5_customer_customer_weight_over_contacts - 338
1.5 - No_shift - 1.5_customer_customer_weight_over_contacts - 409
1.5 - Longer - 1.5_customer_customer_weight_over_contacts - 278
1.5 - No_neighbourhood - 1.5_customer_customer_weight_over_contacts - 401
1.5 - Base - 1.5_customer_customer_weight_over_agents - 649
1.5 - No_shift - 1.5_customer_customer_weight_over_agents - 770
1.5 - Longer - 1.5_custom